In [2]:
from elasticsearch import Elasticsearch, helpers
from rank_bm25 import BM25Okapi
import pandas as pd
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import re
from tqdm import tqdm
import pickle

In [3]:
wiki_dataset = pd.read_json('../data/wikipedia_documents.json',orient='index')
train_dataset = load_from_disk('/opt/ml/data/train_dataset/train').to_pandas()
valid_dataset = load_from_disk('/opt/ml/data/train_dataset/validation').to_pandas()


In [4]:
def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)

    return text

In [4]:
wiki_dataset['text'] = wiki_dataset['text'].apply(preprocess)
#train_dataset['context'] = train_dataset['context'].apply(preprocess)
#valid_dataset['context'] = valid_dataset['context'].apply(preprocess)

In [7]:
wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
train_context = train_dataset['context']
valid_context = valid_dataset['context']
train_query = train_dataset['question']
valid_query = valid_dataset['question']

In [6]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
tokenized_corpus = [tokenizer.tokenize(context) for context in wiki_corpus]
bm25 = BM25Okapi(tokenized_corpus)

INDEX_NAME = "wiki_index"
INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

Token indices sequence length is longer than the specified maximum sequence length for this model (945 > 512). Running this sequence through the model will result in indexing errors


In [7]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : wiki_dataset.iloc[i]['document_id'],
        'title' : wiki_dataset.iloc[i]['title'],
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [8]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [ ]:
es.info()

In [ ]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

In [11]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)


RESPONSE: (60613, [])
106.88828921318054


In [13]:
res = es.search(index=INDEX_NAME, q=train_query[1], size=100)

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [15]:
# reader_data

train_data_with_elastic = {}
error_questions = []
for i in tqdm(range(len(train_query))):
    try:
        res = es.search(index=INDEX_NAME, q=train_query[i], size=100)
        hits = res['hits']['hits']

        top_k_list = []
        for hit in hits:
            ctx = hit['_source']['content']
            top_k_list.append(ctx)
        train_data_with_elastic[train_query[i]] = top_k_list
    except Exception as e:
        error_questions.append([e, train_query[i]])
        train_data_with_elastic[train_query[i]] = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=100)
    
    

100%|██████████| 3952/3952 [03:27<00:00, 19.02it/s]


In [16]:
# reader_data
valid_data_with_elastic = {}
error_valid_questions = []
for i in tqdm(range(len(valid_query))):
    try:
        res = es.search(index=INDEX_NAME, q=valid_query[i], size=100)
        hits = res['hits']['hits']

        top_k_list = []
        for hit in hits:
            ctx = hit['_source']['content']
            top_k_list.append(ctx)
        valid_data_with_elastic[valid_query[i]] = top_k_list
    except Exception as e:
        error_valid_questions.append([e, valid_query[i]])
        valid_data_with_elastic[valid_query[i]] = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=100)
    
    

100%|██████████| 240/240 [00:13<00:00, 17.66it/s]


In [17]:
with open('/opt/ml/data/elastic_train_100.bin', "wb") as file:
    pickle.dump(train_data_with_elastic, file)
with open('/opt/ml/data/elastic_valid_100.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic, file)

In [18]:
test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
test_query = test_dataset['question']


In [19]:
test_data_with_elastic = {}
error_test_questions = []
for i in tqdm(range(len(test_query))):
    try:
        res = es.search(index=INDEX_NAME, q=test_query[i], size=100)
        hits = res['hits']['hits']

        top_k_list = []
        for hit in hits:
            ctx = hit['_source']['content']
            top_k_list.append(ctx)
        test_data_with_elastic[test_query[i]] = top_k_list
    except Exception as e:
        error_test_questions.append([e, test_query[i]])
        test_data_with_elastic[test_query[i]] = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=100)

100%|██████████| 600/600 [00:32<00:00, 18.60it/s]


In [20]:
with open('/opt/ml/data/elastic_retrieval.bin', "wb") as file:
    pickle.dump(test_data_with_elastic, file)

In [5]:
with open('/opt/ml/data/elastic_train_100.bin', "rb") as file:
    train_data_with_elastic = pickle.load(file)
with open('/opt/ml/data/elastic_valid_100.bin', "rb") as file:
    valid_data_with_elastic = pickle.load(file)

In [8]:
for i in range(len(train_query)):
    print(train_query[i])
    print(train_data_with_elastic[train_query[i]][:2])
    break

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
['미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05', '내각(內閣, cabinet)은 행정부의 주요 각료들로 구성되는 국가의 주요기관이다. 의원내각제에서 내각은 수상과 여러 장관으로 조직되는 합의체로, 국가의 행정권을 담당하고 국회에 대한 연대